In [1]:
import pandas as pd
import os 

1.1 Gathering data

In [2]:
os.chdir('D:\Madeline\Documents\BIRC21\Data science')
Data = pd.read_csv('./training.1600000.processed.noemoticon.csv',header = None,encoding = 'ISO-8859-1')

In [3]:
Data.columns = ['Polarity', 'Tweet_ID','Date','Query_username','Username','Tweet_text']
Data.head()

,Polarity,Tweet_ID,Date,Query_username,Username,Tweet_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
# Besoin d'entrainer sur beaucoup moins de tweets (j'en prends que 10 000 ici, 5000 positifs et 5000 négatifs)! 

Datapos = Data[Data['Polarity']==0]
Dataneg = Data[Data['Polarity']==4]

Data_sub = pd.concat([Datapos.sample(5000),Dataneg.sample(5000)], axis=0)
Data_sub.head()

,Polarity,Tweet_ID,Date,Query_username,Username,Tweet_text
739574,0,2265764621,Sun Jun 21 07:15:48 PDT 2009,NO_QUERY,gixywixypixy,@littleboyblues Sana nga we went there nalang...
485178,0,2181045771,Mon Jun 15 11:17:33 PDT 2009,NO_QUERY,veganwrites,@Lindsey_ I was very bummed to hear Obama did ...
699620,0,2254506626,Sat Jun 20 09:51:19 PDT 2009,NO_QUERY,MichaelaMoose,@Ryanthen :o OMG SRSLY? that's so gay (( i lov...
30470,0,1563772448,Mon Apr 20 00:11:56 PDT 2009,NO_QUERY,shmerin,"@snowwahine We totally missed you, you would'v..."
292482,0,1996011865,Mon Jun 01 14:23:48 PDT 2009,NO_QUERY,liamcrook,yep!... definitely sunburnt!


1.2 Dividing between testing and training data

In [5]:
# On va diviser ce dataset en données d'entrainement et données test. J'ai choisi au hasard une proportion 70/30. 

number_tweets = list(Data.shape)[0]
print('On a',number_tweets, 'tweets au total.')
print(set(Data['Polarity']))

On a 1600000 tweets au total.
{0, 4}


In [6]:
from sklearn.model_selection import train_test_split
Data_train, Data_test = train_test_split(Data_sub, test_size=0.3)

In [7]:
Data_train.head()

,Polarity,Tweet_ID,Date,Query_username,Username,Tweet_text
333998,0,2013467609,Tue Jun 02 22:44:54 PDT 2009,NO_QUERY,djmattyg007,@technicolours fair enough. both her and Beckz...
123563,0,1833932850,Mon May 18 00:59:42 PDT 2009,NO_QUERY,sarihta,Insomniaaaaaaaaaa tomorrow ill see 17 again ya...
1540795,4,2180551079,Mon Jun 15 10:37:25 PDT 2009,NO_QUERY,Larissa_134,"why am I on twitter, I never log on...actually"
908681,4,1750930769,Sat May 09 18:15:46 PDT 2009,NO_QUERY,JeremyCShipp,@rorschachsgirl Yeah--those pants are horrible...
1246933,4,1995427780,Mon Jun 01 13:28:28 PDT 2009,NO_QUERY,shampster,@TBone_SB I've been playing with www.wolframal...


2. Preprocessing

In [8]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import warnings
warnings.filterwarnings('ignore')

In [9]:
def prepro(text,i):
    clean=text[i].astype(str).str.replace("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ")
    lowercase=clean.str.lower()
    words=lowercase.apply(word_tokenize)
    stop_words = stopwords.words('english') 
    filtered_words = words.apply(lambda x:[word for word in x if word not in stop_words])
    porter = PorterStemmer()
    text['Tweet_clean']=filtered_words.apply(lambda x:[porter.stem(word) for word in x])
    return text

In [10]:
Data_train_clean=prepro(Data_train,'Tweet_text')
Data_test_clean=prepro(Data_test,'Tweet_text')

In [11]:
Data_train_clean.head()

,Polarity,Tweet_ID,Date,Query_username,Username,Tweet_text,Tweet_clean
333998,0,2013467609,Tue Jun 02 22:44:54 PDT 2009,NO_QUERY,djmattyg007,@technicolours fair enough. both her and Beckz...,"[fair, enough, beckz, hardli, mni, anymor, eit..."
123563,0,1833932850,Mon May 18 00:59:42 PDT 2009,NO_QUERY,sarihta,Insomniaaaaaaaaaa tomorrow ill see 17 again ya...,"[insomniaaaaaaaaaa, tomorrow, ill, see, 17, ya..."
1540795,4,2180551079,Mon Jun 15 10:37:25 PDT 2009,NO_QUERY,Larissa_134,"why am I on twitter, I never log on...actually","[twitter, never, log, actual]"
908681,4,1750930769,Sat May 09 18:15:46 PDT 2009,NO_QUERY,JeremyCShipp,@rorschachsgirl Yeah--those pants are horrible...,"[yeah, pant, horribl, yeah, wife, love, job, l..."
1246933,4,1995427780,Mon Jun 01 13:28:28 PDT 2009,NO_QUERY,shampster,@TBone_SB I've been playing with www.wolframal...,"[sb, play, www, wolframalpha, com, know, answe..."


In [12]:
type(Data_train_clean.iloc[0][6])

list